# Gender Prediction and Age Estimation Using Convolutional Neural Network
In this project we are going to train our CNN model on the UTKFace Dataset.

## Import Libraries
Here we will be importing the required Libraries

In [1]:
from keras.models import Sequential
from keras.models import load_model
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dense,Flatten
from keras.layers import Dropout, BatchNormalization, Input
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import cv2

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Preprocessing the data
Here the path to the dataset is defined and important parameters are specified

In [15]:
path = r'../input/utkface-new/UTKFace'
epoch = 50
lr = 1e-3
img_dims = (200,200,3)
age = []
gender = []
pixels = []

Now, the images are loaded and the info about the age, gender and pixels are added to the specific lists that will be used further in the model

In [3]:
for img in os.listdir(path):
  ages = img.split("_")[0]
  genders = img.split("_")[1]
  img = cv2.imread(str(path)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  pixels.append(np.array(img))
  age.append(np.array(ages))
  gender.append(np.array(genders))
age = np.array(age,dtype=np.int64)
pixels = np.array(pixels)
gender = np.array(gender,np.uint64)

## Splitting the data
The images from the UTKFace dataset is splitted into two sets i.e. training and testing set using the sklearn **train_test_split method**

In [10]:
x_train,x_test,y_train,y_test = train_test_split(pixels,age,random_state=32)
x_train_2,x_test_2,y_train_2,y_test_2 = train_test_split(pixels,gender,random_state=32)

## Building the model 
Now its time to build our model by adding layers to it and creating a neural network
Here, the following layers have been added to build the model.
1. Input Layer - The input layer of a neural network is composed of artificial input neurons, and brings the initial data into the system for further processing by subsequent layers of artificial neurons. 
2. Convolution Layer - Convo layer is sometimes called feature extractor layer because features of the image are get extracted within this layer.
3. Pooling Layer - Pooling layer is used to reduce the spatial volume of input image after convolution. It is used between two convolution layer.
4. Batch Normalisation - Batch normalization is a layer that allows every layer of the network to do learning more independently. It is used to normalize the output of the previous layers.
5. Flatten Layer - Flatten layers allow you to change the shape of the data from a vector of 2d matrixes (or nd matrices really) into the correct format for a dense layer to interpret.
6. Dense Layer - Dense layer is basically used for changing the dimensions of the vector. Dense layers also applies operations like rotation, scaling, translation on the vector.

In [16]:
#building the model
#adding an input layer
input = Input(shape = img_dims)

#adding the convolution layers
conv1 = Conv2D(120,(3,3),activation="relu")(input)
conv2 = Conv2D(100,(3,3),activation="relu")(conv1)

#batch normalization
batch1 = BatchNormalization()(conv2)

#adding a maxpool layer
pool3 = MaxPool2D((2,2))(batch1)

#adding another convolution layer
conv3 = Conv2D(80,(3,3),activation="relu")(pool3)

#batch normalization
batch2 = BatchNormalization()(conv3)

#another maxpool layer to the model
pool4 = MaxPool2D((2,2))(batch2)

#Flattening the output
flt = Flatten()(pool4)

Adding the dense layers

In [17]:
def Add_age(flatten):
    age = Dense(128,activation="relu")(flatten)
    age = Dense(64,activation="relu")(age)
    age = Dense(32,activation="relu")(age)
    age = Dense(1,activation="relu")(age)
    return age
    
def Add_gender(flatten):
    gender = Dense(128,activation="relu")(flatten)
    gender = Dense(80,activation="relu")(gender)
    gender = Dense(64,activation="relu")(gender)
    gender = Dense(32,activation="relu")(gender)
    gender = Dropout(0.5)(gender)
    gender = Dense(2,activation="softmax")(gender)
    return gender

age = Add_age(flt)
gender = Add_gender(flt)

## Compiling and training the model
Now, the final step of compiling and fitting the model for training is performed.

In [18]:
model = Model(inputs=input,
              outputs=[age,gender])

model.compile(optimizer="adam",
              loss=["mse","sparse_categorical_crossentropy"],
              metrics=['mae','accuracy'])

history = model.fit(x_train,[y_train,y_train_2],
                 validation_data=(x_test,[y_test,y_test_2]),
                 epochs=epoch)

Train on 17781 samples, validate on 5927 samples
Epoch 1/50
17781/17781 [==============================] - 121s 7ms/step - loss: 381.4054 - dense_4_loss: 380.0941 - dense_9_loss: 1.3113 - dense_4_mean_absolute_error: 14.2503 - dense_4_acc: 0.0213 - dense_9_mean_absolute_error: 0.5000 - dense_9_acc: 0.6678 - val_loss: 245.4348 - val_dense_4_loss: 244.9948 - val_dense_9_loss: 0.4400 - val_dense_4_mean_absolute_error: 11.4083 - val_dense_4_acc: 0.0290 - val_dense_9_mean_absolute_error: 0.5000 - val_dense_9_acc: 0.8026
Epoch 2/50
17781/17781 [==============================] - 114s 6ms/step - loss: 156.3021 - dense_4_loss: 155.8712 - dense_9_loss: 0.4309 - dense_4_mean_absolute_error: 9.1815 - dense_4_acc: 0.0356 - dense_9_mean_absolute_error: 0.5000 - dense_9_acc: 0.7857 - val_loss: 197.0832 - val_dense_4_loss: 196.7204 - val_dense_9_loss: 0.3629 - val_dense_4_mean_absolute_error: 10.0512 - val_dense_4_acc: 0.0316 - val_dense_9_mean_absolute_error: 0.5000 - val_dense_9_acc: 0.8470
Epoch 3/

## Saving the model
Now, we will save the model so that we can use it in the future.

In [ ]:
model.save("model.h5")

## Plot the Metrics
Here we will plot the metrics i.e. The Accuracy and Loss

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("Accuracy")

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("Loss")